In [1]:
import cv2
import face_recognition
import pickle
import os

In [2]:
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
from firebase_admin import storage

In [3]:
try:
    app = firebase_admin.get_app()
except:
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred, {"databaseURL": "https://faceattendancesystem-b673f-default-rtdb.firebaseio.com/",
                                        "storageBucket": "faceattendancesystem-b673f.appspot.com"})
     

In [4]:
#IMPORTING THE STUDENT IMAGES
folderPath = "C:\\Users\\Priya\\face recognition\\images"
pathList = os.listdir(folderPath)
print(pathList)
imgList = []
studentIds = []
for path in pathList:
    imgList.append(cv2.imread(os.path.join(folderPath,path)))
    studentIds.append(os.path.splitext(path)[0])
    
    # ADD IMAGES TO THE DATABASE
    fileName = f'{folderPath}/{path}'
    bucket = storage.bucket()
    blob = bucket.blob(fileName)
    blob.upload_from_filename(fileName)
    
print(studentIds)

['100.jpg', '123.png', '456.jpg', '789.jpg']
['100', '123', '456', '789']


In [5]:
def findEncodings(imageList):
    encodeList = []
    for img in imageList:
        #the color format is converted from BGR (used by OpenCV) to RGB (used by face_recognition) using cv2.cvtColor.
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        #The face_recognition.face_encodings function is then used to find the face encodings in the image. 
        #Since this function can return multiple encodings (if there are multiple faces in the image), 
        #[0] is used to select the first encoding in the list.
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

print("encoding started..")
encodeListKnown = findEncodings(imgList)
encodeListWithID = [encodeListKnown, studentIds]
print("encoding complete")

file = open("encodeFile.p",'wb')
#Write the `encodeListWithID` object to the file
pickle.dump(encodeListWithID,file)
file.close()
print("file saved")

encoding started..
encoding complete
file saved
